<a href="https://colab.research.google.com/github/KimwWoYoung/kimwooyoung/blob/main/AI_18_%EA%B9%80%EC%9A%B0%EC%98%81_section2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline
from pandas_datareader.data import DataReader
import yfinance as yf
from pandas_datareader import data as pdr
from datetime import datetime


# tech 관련주 4가지 리스트 구성 후 1년치 데이터 추출출
tech_list = ['AAPL', 'GOOG', 'MSFT', 'AMZN']

end = datetime.now()
start = datetime(end.year - 1, end.month, end.day)

for stock in tech_list:
    globals()[stock] = yf.download(stock, start, end)
    

company_list = [AAPL, GOOG, MSFT, AMZN]
company_name = ["APPLE", "GOOGLE", "MICROSOFT", "AMAZON"]

for company, com_name in zip(company_list, company_name):
    company["company_name"] = com_name
    
df = pd.concat(company_list, axis=0)
df.tail(10)

In [ ]:
#데이터 특성 알아보기기
df.describe()

In [ ]:
# 결측치 여부 확인
df.isnull().sum()

In [ ]:
#(AAPL, GOOG, MSFT, AMZN의 Close(종가) 가격 추이 비교 
colors = ['lightsteelblue', 'cornflowerblue', 'royalblue', 'navy']

plt.figure(figsize=(15, 10))
plt.subplots_adjust(top=1.25, bottom=1.2)

for i, company in enumerate(company_list, 1):
    plt.subplot(2, 2, i)
    company['Adj Close'].plot(color=colors[i-1])
    plt.ylabel('Adj Close')
    plt.xlabel(None)
    plt.title(f"Closing Price of {tech_list[i - 1]}")
    
plt.tight_layout()

In [ ]:
# #(AAPL, GOOG, MSFT, AMZN의 거래량 추이 비교 
colors = ['lightsteelblue', 'cornflowerblue', 'royalblue', 'navy']

plt.figure(figsize=(15, 10))
plt.subplots_adjust(top=1.25, bottom=1.2)

for i, company in enumerate(company_list, 1):
    plt.subplot(2, 2, i)
    company['Volume'].plot(color=colors[i-1])
    plt.ylabel('Volume')
    plt.xlabel(None)
    plt.title(f"Sales Volume for {tech_list[i - 1]}")
    
plt.tight_layout()

In [ ]:
#10일, 20일, 50일의 이동평균 시각화화
ma_day = [10, 20, 50]

for ma in ma_day:
    for company in company_list:
        column_name = f"MA for {ma} days"
        company[column_name] = company['Adj Close'].rolling(ma).mean()

fig, axes = plt.subplots(nrows=2, ncols=2)
fig.set_figheight(10)
fig.set_figwidth(15)

AAPL[['Adj Close', 'MA for 10 days', 'MA for 20 days', 'MA for 50 days']].plot(ax=axes[0,0])
axes[0,0].set_title('APPLE')
axes[0,0].legend(loc='lower left')

GOOG[['Adj Close', 'MA for 10 days', 'MA for 20 days', 'MA for 50 days']].plot(ax=axes[0,1])
axes[0,1].set_title('GOOGLE')
axes[0,1].legend(loc='lower left')

MSFT[['Adj Close', 'MA for 10 days', 'MA for 20 days', 'MA for 50 days']].plot(ax=axes[1,0])
axes[1,0].set_title('MICROSOFT')
axes[1,0].legend(loc='lower left')

AMZN[['Adj Close', 'MA for 10 days', 'MA for 20 days', 'MA for 50 days']].plot(ax=axes[1,1])
axes[1,1].set_title('AMAZON')
axes[1,1].legend(loc='lower left')

fig.tight_layout()

In [ ]:
# Daily Return 그래프로 나타내기기
for company in company_list:
    company['Daily Return'] = company['Adj Close'].pct_change()

# Then we'll plot the daily return percentage
fig, axes = plt.subplots(nrows=2, ncols=2)
fig.set_figheight(10)
fig.set_figwidth(15)

AAPL['Daily Return'].plot(ax=axes[0,0], legend=True, linestyle='--', marker='o')
axes[0,0].set_title('APPLE')

GOOG['Daily Return'].plot(ax=axes[0,1], legend=True, linestyle='--', marker='o')
axes[0,1].set_title('GOOGLE')

MSFT['Daily Return'].plot(ax=axes[1,0], legend=True, linestyle='--', marker='o')
axes[1,0].set_title('MICROSOFT')

AMZN['Daily Return'].plot(ax=axes[1,1], legend=True, linestyle='--', marker='o')
axes[1,1].set_title('AMAZON')

fig.tight_layout()

In [ ]:
#  기술 주식 종목들의 일일 수익률
closing_df = pdr.get_data_yahoo(tech_list, start=start, end=end)['Adj Close']

tech_rets = closing_df.pct_change()
tech_rets.head()

In [ ]:
# AAPL	AMZN	GOOG	MSFT 간 관계

sns.pairplot(tech_rets, kind='reg')

In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(2, 2, 1)
sns.heatmap(tech_rets.corr(), annot=True, cmap='Blues')
plt.title('Correlation of stock return')

plt.subplot(2, 2, 2)
sns.heatmap(closing_df.corr(), annot=True, cmap='Blues')
plt.title('Correlation of stock closing price')

In [ ]:
rets = tech_rets.dropna()

area = np.pi * 20

plt.figure(figsize=(10, 8))
plt.scatter(rets.mean(), rets.std(), s=area)
plt.xlabel('Expected return')
plt.ylabel('Risk')

for label, x, y in zip(rets.columns, rets.mean(), rets.std()):
    plt.annotate(label, xy=(x, y), xytext=(50, 50), textcoords='offset points', ha='right', va='bottom', 
                 arrowprops=dict(arrowstyle='-', color='blue', connectionstyle='arc3,rad=-0.3'))

In [ ]:
# Get the stock quote
df = pdr.get_data_yahoo('GOOG', start='2013-04-11', end=datetime.now())
# Show teh data
df

In [ ]:
plt.figure(figsize=(16,6))
plt.title('Close Price GOOG')
plt.plot(df['Close'])
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.show()

In [ ]:
# Close 컬럼 data에 저장장
data = df.filter(['Close'])
dataset = data.values
# 시계열 데이터이기 때문에 순서대로 앞에 80퍼를 train 데이터로 가져온다
training_data_len = int(np.ceil( len(dataset) * .80 ))
training_data_len
# Scale the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(scaled_data[:-60], scaled_data[60:], test_size=0.2, shuffle=False)

In [ ]:
# Close 컬럼 data에 저장장
data = df.filter(['Close'])
dataset = data.values

# 스케일링
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

# training data set 생성 
train_data = scaled_data[0:int(training_data_len), :]
x_train = []
y_train = []
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
        
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1]))

#  models 학습하기
models = [("Linear Regression", LinearRegression()), 
          ("Ridge CV", RidgeCV(alphas=(0.001,0.1,1), cv=3)), 
          ("Lasso CV", LassoCV(alphas=(0.001,0.1,1), cv=5)),
          ("Random Forest", RandomForestRegressor(n_estimators=1000, max_depth=5, min_samples_split=2, random_state=0)), 
          ("XGB Regressor", XGBRegressor(n_estimators=1000, max_depth=5, learning_rate=0.01, random_state=0))]
for model_name, model in models:
    model.fit(x_train, y_train)

# testing data set 생성
test_data = scaled_data[training_data_len - 60: , :]
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1]))

# models predicted price values  구하기
predictions_list = []
for model_name, model in models:
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions.reshape(-1, 1))
    predictions_list.append(predictions)

# models 평가
for i, (model_name, model) in enumerate(models):
    mae = mean_absolute_error(y_test, predictions_list[i])
    mse = mean_squared_error(y_test, predictions_list[i])
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions_list[i])
    print(f"{model_name}:")
    print(f"MAE: {mae:.2f}")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R2 Score: {r2:.2f}\n")

# baseline predictions
baseline_predictions = np.full(shape=(len(y_test), 1), fill_value=y_train.mean())

#  baseline predictions 평가
baseline_mae = mean_absolute_error(y_test, baseline_predictions)
baseline_mse = mean_squared_error(y_test, baseline_predictions)
baseline_rmse = np.sqrt(baseline_mse)
baseline_r2 = r2_score(y_test, baseline_predictions)

print(f"Baseline MAE: {baseline_mae:.2f}")
print(f"Baseline MSE: {baseline_mse:.2f}")
print(f"Baseline RMSE: {baseline_rmse:.2f}")
print(f"Baseline R2 Score: {baseline_r2:.2f}\n")

In [ ]:
# RidgeCV model
ridge_model = RidgeCV(alphas=(0.001,0.1,1), cv=3)
ridge_model.fit(x_train, y_train)

#feature importances
feature_importances = ridge_model.coef_
feature_importances

In [ ]:
# Ridge CV feature importance
ridge = models[1][1]
coef = ridge.coef_
features = []
for i in range(60):
    features.append(f"lag_{i+1}")
    
importance = pd.DataFrame({"Feature": features, "Importance": coef})
importance = importance.set_index("Feature")
importance = importance.abs().sort_values(by="Importance", ascending=False).head(10)

# feature importances 그래프
importance.plot(kind="barh")
plt.title("Ridge CV Feature Importances (Top 10)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()

In [ ]:
# Ridge CV
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions_list[1]

plt.figure(figsize=(16,6))
plt.title('Ridge CV')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
# 예측 가격와 실제 가격 비교교
valid

In [ ]:
# RandomForestRegressor
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions_list[3]

plt.figure(figsize=(16,6))
plt.title('RandomForestRegressor')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
# XGBRegressor 
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions_list[4]

plt.figure(figsize=(16,6))
plt.title('XGBRegressor')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()